## Setup

#  Overview



In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import MatplotlibDeprecationWarning

# this in general not advised, but for this specific notebook
# I'd like to ignore some deprecation warnings by matplotlib
import warnings
warnings.filterwarnings(
    "ignore", category=MatplotlibDeprecationWarning
)

In [2]:
import astropy.units as u
from astropy.coordinates import SkyCoord

## 1. Event Lists

Almost any high-level gamma-ray data analysis starts with the raw measured counts data, which is stored in event lists. In Gammapy event lists are represented by the `EventList` class, which can be imported like so:

In [3]:
from gammapy.data import EventList

Let's created one by reading the Fermi-LAT 3FHL event list:

In [7]:
events_3fhl = EventList.read("data/aerie-apps_bin1-9__allsky_1024_i2.5_r0.0_p1.0_fc2Flux_negSigMap.fits")

KeyError: "Extension 'EVENTS' not found."

In [11]:
from astropy.table import Table
table = Table.read("data/aerie-apps_bin1-9__allsky_1024_i2.5_r0.0_p1.0_fc2Flux_negSigMap.fits",  "extension")
table



significance,flux,flux_lower_bound,flux_upper_bound
float32[1024],float32[1024],float32[1024],float32[1024]
-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30
-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30
-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30
-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30
-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30
-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30
-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30
-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30
-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30,-1.6375e+30 .. -1.6375e+30


In [12]:
table.colnames

['significance', 'flux', 'flux_lower_bound', 'flux_upper_bound']

The actual data is stored as an [astropy.table.Table](http://docs.astropy.org/en/stable/api/astropy.table.Table.html) object. It can be accessed with the `.table` attribute: 

In [5]:
events_3fhl.table

RUN_ID,EVENT_ID,TYPE,RA,DEC,ENERGY,TIME
None,None,None,deg,deg,GeV,s
float64,float64,float64,float32,float32,float32,float64
101.0,1.0,0.0,5.729578,22.918312,1.1,2450000.5
102.0,2.0,1.0,11.459156,28.64789,2.2,2450000.6
103.0,3.0,0.0,17.188734,34.37747,3.3,2450000.7


You can do *len* over event_3fhl.table to find the total number of events.

In [6]:
len(events_3fhl.table)

3

And we can access any other attribute of the `Table` object as well:

In [16]:
events_3fhl.table.colnames

['RUN_ID', 'EVENT_ID', 'TYPE', 'RA', 'DEC', 'ENERGY', 'TIME']

For convenience we can access the most important event parameters as properties on the `EventList` objects. The attributes will return corresponding Astropy objects to represent the data, such as [astropy.units.Quantity](http://docs.astropy.org/en/stable/api/astropy.units.Quantity.html), [astropy.coordinates.SkyCoord](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) or [astropy.time.Time](http://docs.astropy.org/en/stable/api/astropy.time.Time.html#astropy.time.Time) objects:

In [18]:
events_3fhl.energy.to("MeV")

<Quantity [1100., 2200., 3300.] MeV>

In [19]:
events_3fhl.galactic

<SkyCoord (Galactic): (l, b) in deg
    [(114.41410858, -39.48128015), (121.44594471, -34.20769613),
     (126.99179204, -28.35773228)]>

In [11]:
events_3fhl.time

<Time object: scale='tt' format='mjd' value=[54682.82946153 54682.89243456 54682.89709472 ... 57236.75267735
 57233.37455141 57233.44802852]>

There is also some convenience to plot the events:

In [12]:
events_3fhl.plot_image()

In addition `EventList` provides convenience methods to filter the event lists. One possible use case is to find the highest energy event within a radius of 0.5 deg around the vela position:

In [ ]:
# select all events within a radius of 0.5 deg around center
from regions import CircleSkyRegion

center = SkyCoord("0d", "0d", frame="galactic")
region = CircleSkyRegion(center, radius=0.5 * u.deg)
events_gc_3fhl = events_3fhl.select_region(region)

# sort events by energy
events_gc_3fhl.table.sort("ENERGY")

# and show highest energy photon
events_gc_3fhl.energy[-1].to("GeV")

## 2. Maps

The [gammapy.maps]() sub package contains classes to work with sky images and cubes.

In [ ]:
from gammapy.maps import Map

gc_3fhl = Map.create(
    width=(20 * u.deg, 10 * u.deg),
    skydir=center,
    proj="CAR",
    binsz=0.05 *u.deg,
    map_type="wcs",
    frame="galactic"
)

The image is a `~gammapy.maps.WcsNDMap` object:

In [ ]:
gc_3fhl

The shape of the image is 400 x 200 pixel and it is defined using a cartesian projection in galactic coordinates.

The ``geom`` attribute is a `~gammapy.maps.WcsGeom` object:

In [ ]:
print(gc_3fhl.geom)

Now we can fill the events in the map and plot it:

In [ ]:
gc_3fhl.fill_events(events_3fhl)
gc_3fhl.plot(stretch="sqrt", cmap="inferno");

The ``plot`` method basically calls [plt.imshow](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.imshow), passing the `gc_3fhl.data` attribute but in addition handles axis with world coordinates using [astropy.visualization.wcsaxes](https://docs.astropy.org/en/stable/visualization/wcsaxes/) and defines some defaults for nicer plots (e.g. the colormap 'afmhot').

Now we can also take a look at the `.data` attribute: 

In [ ]:
gc_3fhl.data

That looks familiar! It just an *ordinary* 2 dimensional numpy array,  which means you can apply any known numpy method to it:

In [ ]:
print(f"Total number of counts in the image: {gc_3fhl.data.sum():.0f}")

In [ ]:
from gammapy.maps import MapAxis

In [ ]:
energy_axis = MapAxis.from_energy_bounds(
    energy_min="10 GeV", energy_max="2 TeV", nbin=5
)

In [ ]:
print(energy_axis)

In [ ]:
gc_3fhl_cube = Map.create(
    width=(20 * u.deg, 10 * u.deg),
    skydir=center,
    proj="CAR",
    binsz=0.05 *u.deg,
    map_type="wcs",
    frame="galactic",
    axes=[energy_axis]
)

In [ ]:
print(gc_3fhl_cube)

In [ ]:
gc_3fhl_cube.fill_events(events_3fhl)

To make the structures in the image more visible we will smooth the data using a Gaussian kernel.

In [ ]:
gc_3fhl_cube_smoothed = gc_3fhl_cube.smooth(
    kernel="gauss", width=0.1 * u.deg
)

To visualise the data cube we can use interactive plotting:

In [ ]:
gc_3fhl_cube_smoothed.plot_interactive(cmap="inferno")

Or plot the image in energy bands as a grid:

In [ ]:
gc_3fhl_cube_smoothed.plot_grid(
    ncols=3, figsize=(12, 5), cmap="inferno", stretch="sqrt"
);

You can also do a rectangular cutout of a certain region in the image:

In [ ]:
# define center and size of the cutout region
center = SkyCoord(0, 0, unit="deg", frame="galactic")
gc_3fhl_cutout = gc_3fhl_cube_smoothed.cutout(center, 9 * u.deg)
gc_3fhl_cutout.plot_interactive(stretch="sqrt", cmap="inferno");

For a more detailed introduction to `gammapy.maps`, take a look a the [maps.ipynb](https://docs.gammapy.org/0.18.2/tutorials/maps.html) notebook.

## 3. Source catalogs

Gammapy provides a convenient interface to access and work with catalog based data. 

Let's start with importing the 3FHL catalog object from the `~gammapy.catalog` submodule:

In [ ]:
from gammapy.catalog import SourceCatalog3FHL

First we initialize the Fermi-LAT 3FHL catalog and directly take a look at the `.table` attribute:

In [ ]:
fermi_3fhl = SourceCatalog3FHL("data/gll_psch_v13.fit.gz")
fermi_3fhl.table

This looks very familiar again. The data is just stored as an [astropy.table.Table](http://docs.astropy.org/en/stable/api/astropy.table.Table.html#astropy.table.Table) object. We have all the methods and attributes of the `Table` object available. E.g. we can sort the underlying table by `Signif_Avg` to find the top 5 most significant sources:



In [ ]:
# sort table by significance
fermi_3fhl.table.sort("Signif_Avg")

# invert the order to find the highest values and take the top 5
top_five_TS_3fhl = fermi_3fhl.table[::-1][:5]

# print the top five significant sources with association and source class
top_five_TS_3fhl[["Source_Name", "ASSOC1", "ASSOC2", "CLASS", "Signif_Avg"]]

If you are interested in the data of an individual source you can access the information from catalog using the name of the source or any alias source name that is defined in the catalog:

In [ ]:
mkn_421_3fhl = fermi_3fhl["3FHL J1104.4+3812"]

# or use any alias source name that is defined in the catalog
mkn_421_3fhl = fermi_3fhl["Mkn 421"]
print(mkn_421_3fhl.data["Signif_Avg"])

Now we can plot the sources on the image created above:

In [ ]:
_, ax, _ = gc_3fhl.smooth("0.1 deg").plot(
    stretch="sqrt", cmap="inferno"
);

positions = fermi_3fhl.positions
ax.scatter(
    positions.data.lon.deg,
    positions.data.lat.deg,
    transform=ax.get_transform("icrs"),
    color="w",
    marker="x"
)

## 4. Spectral Models and Flux points

In the previous section we learned how access basic data from individual sources in the catalog. Now we will go one step further and explore the full spectral information of sources.

As a first example we will start with the Crab Nebula:

In [ ]:
crab_3fhl = fermi_3fhl["Crab Nebula"]
crab_3fhl_model = crab_3fhl.sky_model()
print(crab_3fhl_model)

In [ ]:
crab_3fhl_spec = crab_3fhl_model.spectral_model

The `crab_3fhl.spectral_model` is an instance of the `PowerLaw2SpectralModel` model, with the parameter values and errors taken from the 3FHL catalog. 

Let's plot the spectral model in the energy range between 10 GeV and 2000 GeV:

In [ ]:
ax_crab_3fhl = crab_3fhl_spec.plot(
    energy_range=[10, 2000] * u.GeV,
)

We assign the return axes object to variable called `ax_crab_3fhl`, because we will re-use it later to plot the flux points on top.

To compute the differential flux at 100 GeV we can simply call the model like normal Python function and convert to the desired units:

In [ ]:
crab_3fhl_spec(100 * u.GeV).to("cm-2 s-1 GeV-1")

Next we can compute the integral flux of the Crab between 10 GeV and 2000 GeV:

In [ ]:
crab_3fhl_spec.integral(
    energy_min=10 * u.GeV, energy_max=2 * u.TeV
).to("cm-2 s-1")

We can easily convince ourself, that it corresponds to the value given in the Fermi-LAT 3FHL catalog:

In [ ]:
crab_3fhl.data["Flux"]

In addition we can compute the energy flux between 10 GeV and 2000 GeV:

In [ ]:
crab_3fhl_spec.energy_flux(
    energy_min=10 * u.GeV, energy_max=2000 * u.GeV
).to("erg cm-2 s-1")

Next we will access the flux points data of the Crab:

In [ ]:
print(crab_3fhl.flux_points)

If you want to learn more about the different flux point formats you can read the specification [here](https://gamma-astro-data-formats.readthedocs.io/en/latest/spectra/flux_points/index.html).

No we can check again the underlying astropy data structure by accessing the `.table` attribute:

In [ ]:
crab_3fhl.flux_points.table

Finally let's combine spectral model and flux points in a single plot and scale with `energy_power=2` to obtain the spectral energy distribution:

In [ ]:
ax = crab_3fhl_spec.plot(
    energy_range=[10, 2000] * u.GeV, energy_power=2
)

ax = crab_3fhl_spec.plot_error(
    energy_range=[10, 2000] * u.GeV,
    energy_power=2,
    facecolor="tab:blue"
)

fp = crab_3fhl.flux_points.to_sed_type("dnde")
fp.plot(ax=ax, energy_power=2);